In [1]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/node-master'       # path to your project on Drive

GIT_USERNAME = "sgsyang" # replace with yours
#GIT_TOKEN = "XXX"           # definitely replace with yours
#GIT_REPOSITORY = "yyy"      # ...nah


drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
GIT_PATH = "https://github.com/sgsyang/node.git"

!mkdir ./temp
!git clone "{GIT_PATH}"
!mv ./temp/* "{PROJECT_PATH}"
!rm -rf ./temp
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/node-master’: File exists
Cloning into 'node'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 0), reused 19 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.
mv: cannot stat './temp/*': No such file or directory
sending incremental file list
README.md
          1,555 100%    0.00k

In [2]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0,1
import os, sys
import time
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
!pip install catboost
!pip install category_encoders
!pip install tensorboardX
!pip install qhoptim



import lib
import torch, torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

experiment_name = 'Catboost'
experiment_name = '{}_{}.{:0>2d}.{:0>2d}_{:0>2d}:{:0>2d}'.format(experiment_name, *time.gmtime()[:5])
print("experiment:", experiment_name)

# DATASETS = {
#     'A9A': fetch_A9A,
#     'EPSILON': fetch_EPSILON,
#     'PROTEIN': fetch_PROTEIN,
#     'YEAR': fetch_YEAR,
#     'HIGGS': fetch_HIGGS,
#     'MICROSOFT': fetch_MICROSOFT,
#     'YAHOO': fetch_YAHOO,
#     'CLICK': fetch_CLICK,
# }

env: CUDA_VISIBLE_DEVICES=0,1
     |████████████████████████████████| 64.8MB 51kB/s 
     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 204kB 4.8MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


experiment: Catboost_2020.05.27_20:21


In [3]:
torch.cuda.get_device_name(device)

'Tesla P100-PCIE-16GB'

In [8]:
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

# 13, 1, 602, 295, 970, 2000, 3000, 10000
random_state = [1337, 133,13, 1, 602, 295, 970, 2000, 3000, 10000]
def Test(random_state):
  data = lib.Dataset("MICROSOFT", random_state=random_state, quantile_transform=True, quantile_noise=1e-3)
  in_features = data.X_train.shape[1]

  mu, std = data.y_train.mean(), data.y_train.std()
  normalize = lambda x: ((x - mu) / std).astype(np.float32)
  data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])

  model = CatBoostRegressor(objective = 'RMSE',task_type="GPU",
                           devices='0:1')
  model.fit(data.X_train,data.y_train)

  # valid_preds = model.predict(data.X_valid)
  test_preds = model.predict(data.X_test)
  test_mse = mean_squared_error(data.y_test, test_preds)
  # print('Catboost for Year Prediction')
  # print("MSE of Validation: %f" % (valid_mse))
  # print("MSE of Test: %f" % (test_mse))

  score = test_mse * std ** 2
  print('-----')
  print('The random_state:',random_state)
  print('The Score',score)
  print(f'Test{random_state} is Over!!!')
  return score, random_state


score_results = [] 
key_results = []


for i in random_state:
  score, random_state = Test(i)
  score_results.append(score)
  key_results.append(random_state)




100%|██████████| 724668170/724668170 [00:19<00:00, 37696134.50it/s]


100%|██████████| 241649037/241649037 [00:07<00:00, 33380648.74it/s]


100%|██████████| 3974680/3974680 [00:00<00:00, 13042899.07it/s]


100%|██████████| 978092/978092 [00:00<00:00, 22448113.49it/s]


Learning rate set to 0.187067
0:	learn: 0.9800067	total: 12ms	remaining: 12s
1:	learn: 0.9652796	total: 22.3ms	remaining: 11.1s
2:	learn: 0.9547695	total: 33.9ms	remaining: 11.3s
3:	learn: 0.9467365	total: 45.6ms	remaining: 11.3s
4:	learn: 0.9409317	total: 56.6ms	remaining: 11.3s
5:	learn: 0.9364580	total: 67.7ms	remaining: 11.2s
6:	learn: 0.9325204	total: 78.1ms	remaining: 11.1s
7:	learn: 0.9294260	total: 88.9ms	remaining: 11s
8:	learn: 0.9270094	total: 99.2ms	remaining: 10.9s
9:	learn: 0.9248702	total: 110ms	remaining: 10.9s
10:	learn: 0.9231284	total: 120ms	remaining: 10.8s
11:	learn: 0.9213285	total: 131ms	remaining: 10.8s
12:	learn: 0.9199101	total: 141ms	remaining: 10.7s
13:	learn: 0.9186211	total: 152ms	remaining: 10.7s
14:	learn: 0.9174814	total: 161ms	remaining: 10.6s
15:	learn: 0.9165224	total: 171ms	remaining: 10.5s
16:	learn: 0.9156587	total: 179ms	remaining: 10.4s
17:	learn: 0.9146116	total: 190ms	remaining: 10.3s
18:	learn: 0.9135786	total: 199ms	remaining: 10.3s
19:	lear

KeyboardInterrupt: ignored

In [9]:
import pprint
Result= dict(zip(key_results,score_results))

print('Microsoft with Catboost')
pprint.pprint(Result, width=1)



Microsoft with Catboost
{}


In [0]:
# random_state = 1337, 133, 13, 1, 602, 295, 970, 2000, 3000, 10000

# random_state = 13

# data = lib.Dataset("YEAR", random_state=random_state, quantile_transform=True, quantile_noise=1e-3)
# in_features = data.X_train.shape[1]

# mu, std = data.y_train.mean(), data.y_train.std()
# normalize = lambda x: ((x - mu) / std).astype(np.float32)
# data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])

# print("mean = %.5f, std = %.5f" % (mu, std))



In [0]:

# from sklearn.metrics import mean_squared_error
# from catboost import CatBoostRegressor
# model = CatBoostRegressor(objective = 'RMSE',task_type="GPU",
#                            devices='0:1')
# model.fit(data.X_train,data.y_train)

# valid_preds = model.predict(data.X_valid)

# valid_mse = mean_squared_error(data.y_valid, valid_preds)
# print("MSE of Validation: %f" % (valid_mse))


In [0]:

# test_preds = model.predict(data.X_test)


# test_mse = mean_squared_error(data.y_test, test_preds)
# print('Catboost for Year Prediction')
# print("MSE of Validation: %f" % (valid_mse))
# print("MSE of Test: %f" % (test_mse))

# print('The random_state:',random_state)
# print('The Score',test_mse * std ** 2)

##Tuned Parameters

In [11]:
!pip install hyperopt
!pip install catboost

import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor


space = {
    'learning_rate':     hp.choice('learning_rate',     np.arange(0.05, 0.31, 0.05)),
    'random_strength':hp.quniform("random_strength", 1,20,1),
    'one_hot_max_size':hp.quniform("one_hot_max_size", 0,25,1),
    'l2_leaf_reg': hp.loguniform('l2_leaf_reg',1 ,10),
    'bagging_temperature':hp.uniform('bagging_temperature', 0, 1),
    'leaf_estimation_iterations':hp.quniform("leaf_estimation_iterations", 1,10,1) 
}

In [13]:
##Automate
def my_tuning(space):
    # reg=CatBoostRegressor(learning_rate=our_space['learning_rate'],
    #                       max_depth = our_space['max_depth'],
    #                       colsample_bylevel = our_space['colsample_bylevel'],
    #                       objective = 'RMSE',task_type="GPU",
    #                       devices='0:1' 
    #                       )
    reg=CatBoostRegressor( learning_rate=space['learning_rate'],
                          random_strength = int(space['random_strength']),
                          one_hot_max_size = int(space['one_hot_max_size']),
                          l2_leaf_reg = space['l2_leaf_reg'],
                          bagging_temperature = space['bagging_temperature'],
                          leaf_estimation_iterations = int(space['leaf_estimation_iterations']),
                          objective = 'RMSE',task_type="GPU",
                          devices='0:1')
        
    reg.fit(data.X_train, data.y_train,verbose = False)

    pred = reg.predict(data.X_test)
    mse= mean_squared_error(data.y_test, pred)
    # print ("test loss:", mse)
    #change the metric if you like
    return {'loss':mse, 'status': STATUS_OK }



score_results = [] 
key_results = []

random_state = [1337, 133,13, 1, 602, 295, 970, 2000, 3000, 10000]
for i in random_state:
  data = lib.Dataset("MICROSOFT", random_state=i, quantile_transform=True, quantile_noise=1e-3)
  in_features = data.X_train.shape[1]
  mu, std = data.y_train.mean(), data.y_train.std()
  normalize = lambda x: ((x - mu) / std).astype(np.float32)
  data.y_train, data.y_valid, data.y_test = map(normalize, [data.y_train, data.y_valid, data.y_test])
  print(f'The dataset with random state {i} is used')

  trials = Trials()
  best = fmin(fn=my_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
  # print('Best parameters',best)
  tuned_loss = trials.best_trial['result']['loss']
  print('Best Loss',tuned_loss)
  final = tuned_loss*std**2
  print('FINAL Error:',final)
  score_results.append(final)
  key_results.append(i)



The dataset with random state 1337 is used
test loss:
0.8238676234460749
test loss:
0.8260122971058467
test loss:
0.8288455740191284
test loss:
0.82480976308791
test loss:
0.8226812119624106
test loss:
0.8300526586398675
test loss:
0.8252459629920313
test loss:
0.8238796150747271
test loss:
0.8258300038262957
test loss:
0.8277996038908715
test loss:
0.8334789911485418
test loss:
0.825838543524637
test loss:
0.8374204941163242
test loss:
0.8320554992865173
test loss:
0.8266388515632548
test loss:
0.8228446033173452
test loss:
0.826309045103265
test loss:
0.8256697057948803
test loss:
0.8308033821549498
test loss:
0.8250348983535982
test loss:
0.8232971867336407
test loss:
0.8235395742765907
test loss:
0.8239499406800904
test loss:
0.8220852207098602
test loss:
0.8241582169284524
test loss:
0.8236527509994885
test loss:
0.8231977186388519
test loss:
0.8229053271722124
test loss:
0.8224369665766034
test loss:
0.8238456758866063
test loss:
0.822908380371709
test loss:
0.8248151709581186
te

In [14]:
import pprint
Result= dict(zip(key_results,score_results))

print('----')
pprint.pprint(Result, width=1)

mean = sum(score_results) / len(score_results)
print('Mean:',mean)
print('Max:',max(score_results))
print('Min:',min(score_results))
print('Distance to max',max(score_results)-mean)
print('Distance to min:',min(score_results)-mean)

----
{1: 0.556681799073326,
 13: 0.5569713898842842,
 133: 0.5567723302358135,
 295: 0.5568818172069695,
 602: 0.5572271731811677,
 970: 0.5564401513493801,
 1337: 0.5573713661203549,
 2000: 0.5571198992753513,
 3000: 0.5571602016293585,
 10000: 0.5570350573005796}
Mean: 0.5569661185256585
Max: 0.5573713661203549
Min: 0.5564401513493801
Distance to max 0.0004052475946964007
Distance to min: -0.0005259671762783968
